### Libraries & Configurations

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)   # Show all rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.width', None)      # Disable line wrapping
pd.set_option('display.max_colwidth', None) # Show long text fully

import re
import ast

import sqlite3

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Heiti TC'

In [2]:
#-----------Data Paths--------------
FILE_RESERVATION = "Reservation.csv"
FILE_UNORDERED = "UnorderedReservation.csv"

### Load Data & Observe

In [33]:
df_reservation = pd.read_csv(FILE_RESERVATION)
df_unordered = pd.read_csv(FILE_UNORDERED)

/var/folders/48/q847vpdj211fy412ns8j2dnm0000gn/T/ipykernel_98070/2320227500.py:1: DtypeWarning: Columns (3,6,11,27,28,29,30,33,36,39,67) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reservation = pd.read_csv(FILE_RESERVATION)


In [12]:
#稍微了解資料型態
df_reservation.head(n=1)

,Unnamed: 0,__v,_id,acceptanceCheckDeadline,acceptanceNote,address,applyBookedSessionInfo,applyBookedSessionLogs,billProcessParticularStatus,bookedSession,cardConfirmedStatus,chatRoom,couponCode,createdAt,customerAdvice,customizations,defaultCardInfo,defaultPaymentMethod,disabled,discount,discountNum,dispatching,email,envInfo,expectServiceTimes,guidePurchase,guidePurchaseRef,importLogFromCsv,intercom,isAssignedSpecificVenderForSelfConstructedBooking,isDirectlyBuild,isFinalReferralFeeNegative,isHasBill,isMinimumChargeUsing,isProcessReferralFeeFlag,isQuotationConfirmed,isRealUseCouponForPs1,isRenewBookedSession,isSelfBooking,isSendNewOrderNotify,isSendNotify,isServicePackageBooking,isStandard,isVisible,labels,memoImages,mode,moveInAddress,name,notes,notificationFrequency,objectNo,originRef,originalTotal,paymentStageSetting,paymentStatus,phone,processingImages,quotation,refReservations,referralFeeDiff,referralFeeInfo,referrer,renewBookedSessionLogs,reservationListMode,reservationNo,review,satisfySurvey,selfBookingInfo,serviceCapacity,serviceCharge,serviceClassification,serviceClassificationName,servicePackageInfo,services,sinyiReservationId,siteImages,staffNumber,state,stateLogs,stateTimes,subtotal,total,totalDiff,travelingExpense,updatedAt,user,utm,valuationInfo,vatNo,vatTitle,vender,venderStaffs,vendersAssignedPriority,verificationPhone
0,0,0,{'oid': '61b849354b524200308101e2'},NaN,NaN,"{'county': '高雄市', 'details': '民享街 １３６號', 'district': '苓雅區', 'fullAddress': '高雄市苓雅區民享街 １３６號', 'hasElevator': None}",NaN,[],ban,"{""from"":""2021-12-20T16:00:00.000Z"",""to"":""2021-12-21T03:59:59.999Z""}",NaN,NaN,NaN,35:17.2,NaN,[],NaN,cash,False,NaN,0,"{'autoDispatchLogs': array([], dtype=float64), 'declineLogs': array([], dtype=float64), 'manuallyDispatchLogs': array([{'_id': {'oid': '61b8493ef7469b003855eb17'}, 'at': Timestamp('2021-12-14 07:35:26.405000'), 'by': {'oid': '611a12a2e6c7f5004bdaeb82'}, 'note': '', 'requestId': '0f92a140-e214-49ba-b5a8-cb1d1a09233c', 'vender': {'oid': '611c7d95fa048f0033a96169'}, 'venderName': '清潔達人股份有限公司'}],\n dtype=object), 'selfBookingLogs': array([{'_id': {'oid': '61b849354b524200308101ea'}, 'at': Timestamp('2021-12-14 07:35:17.293000'), 'by': {'oid': '611a12a2e6c7f5004bdaeb82'}, 'byName': '張小宛', 'requestId': 'f20fb51e-ab44-4339-9213-f08fa800e350', 'role': 'serviceAdmin'}],\n dtype=object)}","b""'0\x16\x9b\xbbp'\xb7R\x05Mt\xdd\xec\xc0\xaf\xa2\xf5E_%\x1co\xd7\x1c\xbd\x14\x9650\xd0\x7f""","{'appVersion': None, 'browserName': None, 'browserVersion': None, 'deviceModel': None, 'deviceType': 'web', 'origin': 'lsWeb', 'osName': None, 'osVersion': None, 'updatedAt': Timestamp('2021-12-14 07:35:06.704000'), 'userAgent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'}",NaN,NaN,NaN,NaN,"{'conversationId': None, 'lastError': None, 'userId': '61b849347e433ad6b3d7ad14'}",NaN,NaN,False,NaN,NaN,False,False,NaN,False,True,NaN,True,False,True,True,NaN,[],general,[],b'K\xe9f\x00t\xc58S\xcb\xb9\xacp`G\xfd\x92',"[{'_id': {'oid': '61b849354b524200308101e4'}, 'at': Timestamp('2021-12-14 07:35:17.206000'), 'by': {'oid': '61b8492a4b524200308101e1'}, 'isReserveRecord': None, 'merchantOrderNo': None, 'note': '[高雄專委送清潔] AG40578 2021120006\n有水\t有電\t空屋-有傢俱\n2房1廳0.0衛0室 \t25.75\t2人4小時\n如有方便想麻煩將雜物清理\n實際時間請再和同仁討論', 'refReservationAction': None, 'refReservationNos': None, 'requestId': 'f20fb51e-ab44-4339-9213-f08fa800e350', 'role': 'customer', 'type': 'reservationNote', 'variedPrice': None}\n {'_id': {'oid': '622e958c0c84f6004abfd09a'}, 'at': Timestamp('2021-12-14 07:35:26.405000'), 'by': {'oid': '611a12a2e6c7f5004bdaeb82'}, 'isReserveRecord': None, 'merchantOrderNo': None, 'note': '無指派廠商 轉為 清潔達人股份有限公司, 備註: ', 'refReservationAction': None, 'refReservationNos': None, 'requestId': '0f92a140-e214-49ba-b5a8-cb1d1a09233c', 'role': 'serviceAdmin', 'type': 'dispatchNote', 'variedPrice': None}]",{'checkState': 0},NaN,{'coupon': None},4000,"{'1': {'accountsPayable': None, 'applePayme

In [13]:
#稍微了解資料型態
df_unordered.head(n=1)

,Unnamed: 0,__v,_id,acceptanceNote,address,billProcessParticularStatus,chatRoom,completedDate,constructionDate,contactDate,couponCode,createdAt,customerAdvice,customerBudget,customizations,defaultPaymentMethod,disabled,discount,discountNum,dispatching,email,envInfo,expectedCompletedDate,guidePurchase,guidePurchaseRef,intercom,isDirectlyBuild,isFinalReferralFeeNegative,isHasBill,isProcessReferralFeeFlag,isQuotationConfirmed,isSelfBooking,isSendNewOrderNotify,isSendNotify,isStandard,isVisible,labels,memoImages,mode,name,newestStateTimes,notes,noticeAcceptedCount,objectNo,originRef,originalTotal,phone,processingImages,quotation,refUnorderedReservations,referralFeeDiff,referralFeeInfo,referrer,reservationListMode,satisfySurvey,selfBookingInfo,serviceCapacity,serviceCharge,serviceClassification,serviceClassificationName,services,siteImages,specificVenderAssigned,staffNumber,state,stateLogs,subtotal,total,totalDiff,unorderedReservationNo,updatedAt,user,utm,vatNo,vatTitle,vender,vendersAssignedPriority,verificationPhone
0,0,0,{'oid': '632193b7a9c94e0031b8fa7e'},客戶備註：,"{'county': '新北市', 'details': '信義路五段100號', 'district': '萬里區', 'fullAddress': '新北市萬里區信義路五段100號', 'hasElevator': False}",ban,NaN,NaN,NaN,不拘(09:00-18:00),SINYIAG1,41:27.1,NaN,NaN,[],cash,False,NaN,0,"{'manuallyDispatchCancelLogs': None, 'manuallyDispatchLogs': array([], dtype=float64), 'selfBookingLogs': array([], dtype=float64)}","b'""P8g\x9e#\x02\x9b\x05\xefr\xae\x08\x01\x83\xec\xaf\xd0""\xe1\x98\xa9\xf0\x84\xfa\x80,\xf7\xe4\xcd\x1d\x16'","{'deviceType': 'web', 'appVersion': '1.18.0', 'browserName': 'chrome', 'browserVersion': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36', 'deviceModel': 'Win32', 'osName': 'Windows', 'osVersion': '', 'origin': 'lsWeb', 'userAgent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36', 'updatedAt': Timestamp('2022-09-14 08:41:27.099000')}",NaN,NaN,NaN,"{'conversationId': None, 'userId': '6283e6025065406b2f15420e'}",NaN,False,NaN,False,False,False,NaN,True,False,False,[{'oid': '6391a60fc7494f002fd9ee42'} {'oid': '6391a60ffc36400031166ca0'}],[],decoration,b'\x9eI\x94]X\xca\x9a\xa0\xbedp\r\xb9\x92s\x8f',"{'completedTime': None, 'dispatchingTime': None, 'pendingTime': Timestamp('2022-09-14 08:41:27.215000')}","[{'_id': {'oid': '632193b7a9c94e0031b8fa80'}, 'at': Timestamp('2022-09-14 08:41:27.139000'), 'by': {'oid': '620204f826db910031debeec'}, 'isReserveRecord': None, 'note': '', 'refUnorderedReservationAction': None, 'refUnorderedReservationNos': array([], dtype=object), 'requestId': 'af962e33-e812-45af-967d-7ba27fd8522b', 'role': 'customer', 'type': 'unorderedReservationNote'}]",0,NaN,{'coupon': {'oid': '62a1b49216b70f0032daa1c0'}},0,b'5\x1dsE\xde\xfe\x10\xd4\xcf\x92\xb3\x89\xa4B0\xfc',[],NaN,[],[],NaN,NaN,stateUnordered,NaN,NaN,NaN,"{'type': 'rate', 'amount': 0}",{'oid': '63219016a9c94e0031b8f3ee'},室內設計,"[{'_id': {'oid': '632193b7a9c94e0031b8fa7f'}, 'serviceId': {'oid': '63219209ade20d0032445034'}, 'count': 1, 'price': 0, 'basicPrice': 0}]",[],NaN,459239.0,{'oid': '63155bc6664730006ee29c5e'},"[{'_id': {'oid': '632193b7a9c94e0031b8fa81'}, 'at': Timestamp('2022-09-14 08:41:27.215000'), 'by': {'oid': '620204f826db910031debeec'}, 'newInfo': {'clientStateName': '預約服務', 'completedDate': None, 'consoleStateName': '待處理', 'isCancelled': False, 'isCompleted': False, 'isInitial': True}, 'newState': {'oid': '63155bc6664730006ee29c5e'}, 'note': '', 'originalState': {'oid': '63155bc6664730006ee29c5e'}, 'payload': None, 'requestId': 'af962e33-e812-45af-967d-7ba27fd8522b'}]",0,0,0,0000V2,15:06.7,{'oid': '620204f826db910031debeec'},"{'source': '', 'medium': '', 'campaign': '', 'content': '', 'term': ''}",NaN,NaN,NaN,[],NaN


觀察到主要分成三種類別的資料
1. 乾淨的數值或文字 (如:'serviceClassificationName') >> 不需處理

In [14]:
df_reservation['serviceClassificationName'].head(n=5)

0    居家客服專用管道
1    居家客服專用管道
2        優質搬家
3    居家客服專用管道
4    居家客服專用管道
Name: serviceClassificationName, dtype: object

2. 為一個 dictionary 型態的資料 (如: 'address' ) >> 將 key-value pairs 提出為獨立欄位，命名邏輯可為 [原始欄位名稱]_[key]

In [15]:
df_reservation['address'].head(n=2)

0        {'county': '高雄市', 'details': '民享街   １３６號', 'district': '苓雅區', 'fullAddress': '高雄市苓雅區民享街   １３６號', 'hasElevator': None}
1    {'county': '高雄市', 'details': '金門街   ９５號１２樓', 'district': '苓雅區', 'fullAddress': '高雄市苓雅區金門街   ９５號１２樓', 'hasElevator': None}
Name: address, dtype: object

3. 為多個 dictionary 型態的資料 (如: 'customization') >> 獨立成自己的表，以訂單 id ('_id) 來辨識每項對應到哪個訂單

In [16]:
#3. 為多個 dictionary 型態的資料 (如: 'customization') >> 獨立成自己的表，以訂單 id ('_id) 來辨識每項對應到哪個訂單
df_reservation['customizations'].head(n=3)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        []
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

### Data Cleaning Logic

##### 基本資料值處理 funcitons

In [4]:
"""
不管是 dict 還是 list of dict 的欄位，在資料內都是以 str. 的型態儲存，因次需先轉換成對應的 python object
"""

def fix_missing_commas(raw_string):
    """
    注意到 list of dict 缺少 dict 之間的逗號，透過此function將逗號加在適當的位置
    Example: [{...}{...}] ➝ [{...}, {...}]
    """
    # Insert a comma between '}{' only if they are not already separated
    fixed_string = re.sub(r'\}[\s]*\{', '}, {', raw_string)
    return fixed_string


def parse_json_like(str_val):
    """
    將 string 讀成 Python 物件
    """
    try:
        str_val = fix_missing_commas(str_val)

        #若為時間資料的特殊處理
        if "Timestamp(" in str_val:
            str_val = str_val.replace("Timestamp(", "")
            str_val = str_val.replace(")", "")

        return ast.literal_eval(str_val)
    except Exception as e:
        #print(str_val)
        #print(f"Error parsing services : {e}")
        return []

#### 型態 2

dictionary 型態的資料 (如: 'address' ) >> 將 key-value pairs 提出為獨立欄位，命名邏輯可為 [原始欄位名稱]_[key]

In [5]:
#資料處理函數
def expand_dict_like(df, col_name, converted = False, rename = True):
    """
    讀入有 dict 型態的欄位，並回傳key-value pairs 提出後的dataframe 
    converted : 資料是否已轉換為 python objects
    rename : 是否修改欄位名稱為 [原始欄位名稱]_[key]
    """
    if not converted:
        df_parsed = df[col_name].apply(parse_json_like)
    else:
        df_parsed = df[col_name]
        
    df_expanded = df_parsed.apply(lambda x: pd.Series(x))
    try:
        df_expanded.drop([0], axis=1, inplace=True)
    except KeyError:
        pass

    if rename:
        df_expanded.columns = [col_name + "_" + str(key) for key in df_expanded.columns]
    
    return df_expanded 

In [90]:
#範例:以 'address' column 處理為例

print("===處理前===")
print(df_reservation['address'][0])
print("===處理後===")
expand_dict_like(df_reservation, 'address').head(n=1)

===處理前===
{'county': '高雄市', 'details': '民享街   １３６號', 'district': '苓雅區', 'fullAddress': '高雄市苓雅區民享街   １３６號', 'hasElevator': None}
===處理後===


,address_county,address_details,address_district,address_fullAddress,address_hasElevator
0,高雄市,民享街 １３６號,苓雅區,高雄市苓雅區民享街 １３６號,None


#### 型態 3
多個 dictionary 型態包成一個list 的資料 (如: 'customization') >> 獨立成自己的表，以訂單 id ('_id) 來辨識每項對應到哪個訂單

In [6]:
#資料處理函數
def extract_dict_lst_like(df, col_name, id_col):
    """
        col_name : 要處理的欄位
        id_col : 與主表連接的 id 

        回傳獨立的 dataframe
    """

    #提出需要的欄位，並將重點欄位轉換為python物件
    df_current = df[[id_col, col_name]]
    df_current.columns = ['order_id', col_name] #以便區分訂單ID 以及這個表自己的id 
    df_current[col_name] = df_current[col_name].apply(parse_json_like)

    #將 list of dicts 拆成多個row
    df_exploded = df_current.explode(col_name)

    #將每個 row 的 dict 獨立成自己的欄位 (與型態 2 一樣)
    detail_expanded = expand_dict_like(df_exploded, col_name, converted=True, rename=False)
    try:
        detail_expanded.drop(0, axis=1, inplace=True)
    except KeyError: #若沒有多餘的0欄位
        pass
    
    df_final = pd.concat([df_exploded['order_id'], detail_expanded], axis = 1)

    return df_final

In [21]:
#範例:以 'customization' column 處理為例
print("===處理前===")
print(df_reservation['customizations'][2])
print("===處理後===")
extract_dict_lst_like(df_reservation, 'customizations', '_id').head()

#注意到清出來的表裡面可能又會有要清的dictionary，那就再用型態2的function 清，以此類推

===處理前===
[{'_id': {'oid': '61b84a734b524200308101f9'}, 'addCost': 0, 'count': 1, 'customizationId': None, 'customizedValue': None, 'expandedItem': None, 'itemId': {'oid': '615297ab48781900308dbb53'}, 'name': '配送到遷出地址(下一頁填寫)', 'subtotal': 0, 'title': '紙箱配送(含加購及贈送)'}
 {'_id': {'oid': '61b84a734b524200308101fa'}, 'addCost': 0, 'count': 1, 'customizationId': None, 'customizedValue': None, 'expandedItem': None, 'itemId': {'oid': '611325eb6ec84b0032f28e89'}, 'name': '紙箱', 'subtotal': 0, 'title': '其他加購項目(另報價)'}]
===處理後===


/var/folders/48/q847vpdj211fy412ns8j2dnm0000gn/T/ipykernel_92575/1033596803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current[col_name] = df_current[col_name].apply(parse_json_like)


,order_id,_id,addCost,count,customizationId,customizedValue,expandedItem,itemId,name,subtotal,title
0,{'oid': '61b849354b524200308101e2'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'oid': '61b849ba4b524200308101ed'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'oid': '61b84a734b524200308101f8'},{'oid': '61b84a734b524200308101f9'},0.0,1.0,None,None,None,{'oid': '615297ab48781900308dbb53'},配送到遷出地址(下一頁填寫),0.0,紙箱配送(含加購及贈送)
2,{'oid': '61b84a734b524200308101f8'},{'oid': '61b84a734b524200308101fa'},0.0,1.0,None,None,None,{'oid': '611325eb6ec84b0032f28e89'},紙箱,0.0,其他加購項目(另報價)
3,{'oid': '61b84e48f7469b003855eb41'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleaning

#### 一般訂單清洗

##### 欄位分類:分成 1、2、3 型

做法:將 schema 丟給 chatGPT，請他分類成一個 dictionary

Prompt:
``` 

Write a python dictionary for me with three keys: "type_1", "type_2" and "type_3", each containing the column name that corresponds to the type of data within it.

type_1 : clean text or number
type_2: dictionary-like item
type_3: list of dictionary like

Does this make sense? If yes, I'll give you the schema of my dataset along with the columns I want sorted. 
You should ask me for clarification on any abiguous columns. 

```
(確認後直接貼上 schema 跟 所有欄位名稱)

In [7]:
#要分類的欄位名稱，可給chatGPT參考
df_reservation.columns

Index(['Unnamed: 0', '__v', '_id', 'acceptanceCheckDeadline', 'acceptanceNote',
       'address', 'applyBookedSessionInfo', 'applyBookedSessionLogs',
       'billProcessParticularStatus', 'bookedSession', 'cardConfirmedStatus',
       'chatRoom', 'couponCode', 'createdAt', 'customerAdvice',
       'customizations', 'defaultCardInfo', 'defaultPaymentMethod', 'disabled',
       'discount', 'discountNum', 'dispatching', 'email', 'envInfo',
       'expectServiceTimes', 'guidePurchase', 'guidePurchaseRef',
       'importLogFromCsv', 'intercom',
       'isAssignedSpecificVenderForSelfConstructedBooking', 'isDirectlyBuild',
       'isFinalReferralFeeNegative', 'isHasBill', 'isMinimumChargeUsing',
       'isProcessReferralFeeFlag', 'isQuotationConfirmed',
       'isRealUseCouponForPs1', 'isRenewBookedSession', 'isSelfBooking',
       'isSendNewOrderNotify', 'isSendNotify', 'isServicePackageBooking',
       'isStandard', 'isVisible', 'labels', 'memoImages', 'mode',
       'moveInAddress', 'name

In [8]:
#chatGPT 分類

column_type_map = {
    "type_1": [
        "acceptanceCheckDeadline", "acceptanceNote", "couponCode", "createdAt",
        "customerAdvice", "defaultPaymentMethod", "disabled", "discountNum",
        "email", "expectServiceTimes", "isAssignedSpecificVenderForSelfConstructedBooking",
        "isDirectlyBuild", "isFinalReferralFeeNegative", "isHasBill",
        "isMinimumChargeUsing", "isProcessReferralFeeFlag", "isQuotationConfirmed",
        "isRealUseCouponForPs1", "isRenewBookedSession", "isSelfBooking",
        "isSendNewOrderNotify", "isSendNotify", "isServicePackageBooking",
        "isStandard", "isVisible", "mode", "name",
        "objectNo", "originalTotal", "phone", "referrer", "reservationListMode",
        "reservationNo", "satisfySurvey", "serviceCapacity",
        "serviceClassificationName", "sinyiReservationId", "staffNumber", "state",
        "subtotal", "total", "totalDiff", "travelingExpense", "updatedAt",
        "vatNo", "vatTitle", "verificationPhone"
    ],
    "type_2": [
        "_id", "user", "vender", "quotation", "serviceClassification",
        "chatRoom", "guidePurchase", "guidePurchaseRef", "serviceCharge", "review",
        "defaultCardInfo", "cardConfirmedStatus", "valuationInfo", "originRef",
        "discount", "paymentStatus", "bookedSession", "address",
        "applyBookedSessionInfo", "billProcessParticularStatus", "dispatching",
        "envInfo", "paymentStageSetting", "selfBookingInfo", "servicePackageInfo",
        "stateTimes", "intercom", "utm", "notificationFrequency"
    ],
    "type_3": [
        "applyBookedSessionLogs", "customizations", "labels", "memoImages",
        "moveInAddress", "notes", "processingImages", "refReservations",
        "referralFeeDiff", "renewBookedSessionLogs", "services", "siteImages",
        "stateLogs", "venderStaffs", "vendersAssignedPriority"
    ]
}

##### 依序清理第一、第二、第三類型欄位

In [31]:
#先將全部都是缺失值的欄位drop掉
df_reservation.dropna(how='all', axis=1, inplace=True)

In [32]:
#將 type 1 直接存進清理後資料表
df_cleaned = df_reservation[column_type_map['type_1']]

KeyError: "['customerAdvice'] not in index"

In [15]:
#將 type 2 一個一個拆出來加到清理後資料表 (約跑4~6分鐘)

error_columns = [] #儲存處理時出錯的欄位

for i, col in enumerate(column_type_map['type_2']):
    try:
        col_expanded = expand_dict_like(df_reservation, col)
        df_cleaned = pd.concat([df_cleaned, col_expanded], axis=1)
        print(f"{i+1} of {len(column_type_map['type_2'])} processed -- {col}")
    except:
        print(f"Problem processing {col} ... added to error_columns list")
        error_columns.append(col)

1 of 29 processed -- _id
2 of 29 processed -- user
3 of 29 processed -- vender
4 of 29 processed -- quotation
5 of 29 processed -- serviceClassification
6 of 29 processed -- chatRoom
7 of 29 processed -- guidePurchase
8 of 29 processed -- guidePurchaseRef
9 of 29 processed -- serviceCharge
10 of 29 processed -- review
11 of 29 processed -- defaultCardInfo
12 of 29 processed -- cardConfirmedStatus
13 of 29 processed -- valuationInfo
14 of 29 processed -- originRef
15 of 29 processed -- discount
16 of 29 processed -- paymentStatus
17 of 29 processed -- bookedSession
18 of 29 processed -- address
19 of 29 processed -- applyBookedSessionInfo
20 of 29 processed -- billProcessParticularStatus
21 of 29 processed -- dispatching
22 of 29 processed -- envInfo
23 of 29 processed -- paymentStageSetting
24 of 29 processed -- selfBookingInfo
25 of 29 processed -- servicePackageInfo
26 of 29 processed -- stateTimes
27 of 29 processed -- intercom
28 of 29 processed -- utm
29 of 29 processed -- notific

In [17]:
#=====初步清理掉全部都是na的欄位========
df_cleaned.dropna(how='all', axis=1, inplace=True)

In [23]:
#====遞迴將清出來還是dict的繼續展開，直到所有欄位都是純質======
error_columns = []
loops = 0
while True:
    loops += 1
    flag = 0
    print(f"Loop : {loops}")
    for col in df_cleaned.columns:
        
        sample = df_cleaned[col].dropna().iloc[0] #尋找第一個非n/a的值，以此辨識是否需要展開

        if type(sample) == type(dict()):
            print(f"Processing dict : {col}")
            flag = 1
            try:
                col_expanded = expand_dict_like(df_cleaned, col, converted=True)
                df_cleaned = pd.concat([df_cleaned, col_expanded], axis=1)
                df_cleaned.drop(col, axis=1, inplace=True)
            except:
                print(f"Problem processing {col} ... added to error_columns list")
                error_columns.append(col)

    if flag == 0:
        break

Loop : 1
Processing dict : defaultCardInfo_spgatewayCardInfo
Processing dict : originRef_coupon
Processing dict : applyBookedSessionInfo_bookedSession
Processing dict : selfBookingInfo_copyFrom
Processing dict : selfBookingInfo_copyOpt
Processing dict : selfBookingInfo_fromUserBookedReservation
Processing dict : servicePackageInfo_copyParams
Processing dict : intercom_lastError
Loop : 2
Processing dict : servicePackageInfo_copyParams_copyService
Processing dict : servicePackageInfo_copyParams_frequency
Loop : 3


In [24]:
print(f"資料總筆數: {df_cleaned.shape[0]}")

資料總筆數: 27813


In [26]:
#匯入資料庫
conn = sqlite3.connect('xinyi_db.sqlite')
cursor = conn.cursor()

df_cleaned.to_sql("reservation_main", conn)

27813

In [28]:
#確認缺失值狀況
missing_counts = df_cleaned.isna().sum().sort_values(ascending=False)
missing_counts = missing_counts[missing_counts > 0]  # Only show columns with missing values
missing_counts

"""
#可考慮將缺失值大於 60% 的欄位直接 drop 掉
drop_columns = missing_counts[missing_counts > df_cleaned.shape[0]*0.6]
df_cleaned.drop(list(drop_columns.index), axis=1, inplace=True)
"""

'\n可考慮將缺失值大於 60% 的欄位直接 drop 掉\ndrop_columns = missing_counts[missing_counts > df_cleaned.shape[0]*0.6]\ndf_cleaned.drop(list(drop_columns.index), axis=1, inplace=True)\n'

In [95]:
#存為csv檔案
df_cleaned.to_excel("reservation_cleaned.xlsx", sheet_name='main', engine = 'xlsxwriter')

In [29]:
#處理 type-3 資料 (約跑3~5分鐘)
subDF_dict = {} #儲存獨立出的附表 
error_cols = [] #儲存處理時出錯的欄位

for i, col in enumerate(column_type_map["type_3"]):
    try:
        expanded_df = extract_dict_lst_like(df_reservation, col, "_id")
        subDF_dict[col] = expanded_df
        print(f"{i+1} of {len(column_type_map['type_3'])} processed -- {col}")
        
    except:
        print(f"Problem processing {col} ... added to error_columns list")
        error_cols.append(col)

/var/folders/48/q847vpdj211fy412ns8j2dnm0000gn/T/ipykernel_98070/1033596803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current[col_name] = df_current[col_name].apply(parse_json_like)


1 of 15 processed -- applyBookedSessionLogs


/var/folders/48/q847vpdj211fy412ns8j2dnm0000gn/T/ipykernel_98070/1033596803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current[col_name] = df_current[col_name].apply(parse_json_like)


2 of 15 processed -- customizations


/var/folders/48/q847vpdj211fy412ns8j2dnm0000gn/T/ipykernel_98070/1033596803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current[col_name] = df_current[col_name].apply(parse_json_like)


...

13 of 15 processed -- stateLogs


/var/folders/48/q847vpdj211fy412ns8j2dnm0000gn/T/ipykernel_98070/1033596803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current[col_name] = df_current[col_name].apply(parse_json_like)


14 of 15 processed -- venderStaffs


/var/folders/48/q847vpdj211fy412ns8j2dnm0000gn/T/ipykernel_98070/1033596803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current[col_name] = df_current[col_name].apply(parse_json_like)


15 of 15 processed -- vendersAssignedPriority


In [30]:
print("獨立出的 Dataframes:")
list(subDF_dict.keys())

獨立出的 Dataframes:


['applyBookedSessionLogs',
 'customizations',
 'labels',
 'memoImages',
 'moveInAddress',
 'notes',
 'processingImages',
 'refReservations',
 'referralFeeDiff',
 'renewBookedSessionLogs',
 'services',
 'siteImages',
 'stateLogs',
 'venderStaffs',
 'vendersAssignedPriority']

In [34]:
for sub_df in subDF_dict.keys():
    print("\n===\n")
    print(sub_df)
    df = subDF_dict[sub_df]
    print("Total Rows: " + str(df.shape[0]))
    #確認缺失值狀況
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_counts = missing_counts[missing_counts > 0]  # Only show columns with missing values
    print(missing_counts)
    print("\n===\n")

"""
keep = ["customizations", "moveInAddress", "notes", "renewBookedSessionLogs", "services", "stateLogs"]
"""


===

applyBookedSessionLogs
Total Rows: 27813
applyBookedSession    27800
at                    27800
beforeEdit            27800
by                    27800
byName                27800
requestId             27800
dtype: int64

===


===

customizations
Total Rows: 40444
expandedItem       40285
customizedValue    34847
customizationId    24807
_id                17759
addCost            17759
count              17759
itemId             17759
name               17759
subtotal           17759
title              17759
dtype: int64

===


===

labels
Total Rows: 28254
oid    25287
dtype: int64

===


===

memoImages
Total Rows: 28352
oid    27611
dtype: int64

===


===

moveInAddress
Total Rows: 28078
hasElevator    23598
county         21233
details        21233
district       21233
fullAddress    21233
type           21233
dtype: int64

===


===

notes
Total Rows: 46585
isReserveRecord         46585
refReservationAction    46585
refReservationNos       46585
merchantOrderNo         4

'\nkeep = ["customizations", "moveInAddress", "notes", "renewBookedSessionLogs", "services", "stateLogs"]\n'

In [68]:
keep = ["customizations", "moveInAddress", "notes", "renewBookedSessionLogs", "services", "stateLogs"]

In [38]:
for sub_table in subDF_dict.keys():
    print("\n=====\n")
    print(f"Cleaning sub_table {sub_table}")

    df_cleaned = subDF_dict[sub_table]
    df_cleaned.dropna(how = "all", axis = 1, inplace=True)
    cols = df_cleaned.columns

    loops = 0
    while True:
        loops += 1
        flag = 0
        print(f"Loop : {loops}")
        for col in df_cleaned.columns:
            
            sample = df_cleaned[col].dropna().iloc[0] #尋找第一個非n/a的值，以此辨識是否需要展開

            if type(sample) == type(dict()):
                print(f"Processing dict : {col}")
                flag = 1
                try:
                    col_expanded = expand_dict_like(df_cleaned, col, converted=True)
                    df_cleaned = pd.concat([df_cleaned, col_expanded], axis=1)
                    df_cleaned.drop(col, axis=1, inplace=True)
                except:
                    print(f"Problem processing {col} ... added to error_columns list")
                    error_columns.append(col)

        if flag == 0:
            break

    """
    with pd.ExcelWriter("reservation_cleaned.xlsx", mode="a", engine="openpyxl", if_sheet_exists="new") as writer:
        df_cleaned = df_cleaned.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
    
        df_cleaned.to_excel(writer, sheet_name=sub_table, index=False)
    """
    df_cleaned.to_sql(sub_table, conn)
    print("Subtable saved to sql db.")
    print("\n=====\n")



=====

Cleaning sub_table applyBookedSessionLogs
{'oid': '61b849354b524200308101e2'}
<class 'str'>
{'from': '2024-07-18 00:00:00', 'to': '2024-07-18 00:00:00'}
<class 'dict'>
2024-07-08 02:53:06.985000
<class 'str'>
{'from': '2024-07-16 10:00:00', 'to': '2024-07-16 10:00:59.999000'}
<class 'dict'>
{'oid': '611a0093c419730031308801'}
<class 'dict'>
周正庭
<class 'str'>
a640b450-5a2e-4a33-b6e3-4ab47e94fee2
<class 'str'>
Subtable saved to sql db.

=====


=====

Cleaning sub_table customizations
{'oid': '61b849354b524200308101e2'}
<class 'str'>
{'oid': '61b84a734b524200308101f9'}
<class 'dict'>
0.0
<class 'numpy.float64'>
1.0
<class 'numpy.float64'>
622185d6887e090035be51cf
<class 'str'>
新北市三重區三民街272巷
<class 'str'>
word
<class 'str'>
{'oid': '615297ab48781900308dbb53'}
<class 'dict'>
配送到遷出地址(下一頁填寫)
<class 'str'>
0.0
<class 'numpy.float64'>
紙箱配送(含加購及贈送)
<class 'str'>
Subtable saved to sql db.

=====


=====

Cleaning sub_table labels
{'oid': '61b849354b524200308101e2'}
<class 'str'>
62aa918f

ProgrammingError: Error binding parameter 10: type 'dict' is not supported

In [92]:
df_cleaned.head()

,order_id_oid,_id_oid,addCost,count,customizationId,customizedValue,expandedItem,itemId_oid,name,subtotal,title
0,61b849354b524200308101e2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,61b849ba4b524200308101ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61b84a734b524200308101f8,61b84a734b524200308101f9,0.0,1.0,None,None,None,615297ab48781900308dbb53,配送到遷出地址(下一頁填寫),0.0,紙箱配送(含加購及贈送)
2,61b84a734b524200308101f8,61b84a734b524200308101fa,0.0,1.0,None,None,None,611325eb6ec84b0032f28e89,紙箱,0.0,其他加購項目(另報價)
3,61b84e48f7469b003855eb41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
